In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
#Parámetro para el entorno
dbutils.widgets.text("ENV", "dev")
ENV = dbutils.widgets.get("ENV").strip().lower()

if ENV not in ("dev", "prod"):
    raise ValueError("ENV debe ser 'dev' o 'prod'")

In [0]:
#Configuración de entorno para trabajar
if ENV == "dev":
    CATALOG = "dev-adventureworks"
    ADLS_ACCOUNT = "adsldevadventureworks"
else:
    CATALOG = "prod-adventureworks"
    ADLS_ACCOUNT = "adslprodadventureworks"

print("ENV:", ENV)
print("CATALOG:", CATALOG)
print("ADLS_ACCOUNT:", ADLS_ACCOUNT)

In [0]:
#Usamos la base de datos y el schema para poder trabajar
spark.sql(f"USE CATALOG `{CATALOG}`")
spark.sql(F"USE SCHEMA silver_schema")

In [0]:
# Esta es la tabla Silver "normal" (estado actual del producto)
df_product = spark.table("product_silver")

display(df_product)

In [0]:
# Clave primaria del producto
PK = "ProductID"   # cambia si tu columna se llama distinto


In [0]:
# Columnas de negocio que, si cambian, generan una nueva versión
BUSINESS_COLS = [
    c for c in df_product.columns 
    if c not in [PK]
]


In [0]:
from pyspark.sql.functions import col, sha2, concat_ws, current_timestamp, lit

df_src = (
    df_product
    # Hash: si algo cambia en las columnas de negocio, el hash cambia
    .withColumn(
        "record_hash",
        sha2(
            concat_ws("||", *[col(c).cast("string") for c in BUSINESS_COLS]),
            256
        )
    )
    # Columnas SCD2
    .withColumn("effective_from", current_timestamp())   # desde cuándo es válido
    .withColumn("effective_to", lit(None).cast("timestamp"))  # hasta cuándo
    .withColumn("is_current", lit(True))  # registro vigente
)

df_src.createOrReplaceTempView("src_product")

display(df_src)

#El record_hash sirve para saber si algo cambió
#No comparamos columna por columna, solo el hash


In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver_schema.product_silver_scd2
USING DELTA
AS
SELECT
  *,
  CAST(NULL AS STRING)     AS record_hash,
  CAST(NULL AS TIMESTAMP)  AS effective_from,
  CAST(NULL AS TIMESTAMP)  AS effective_to,
  CAST(false AS BOOLEAN)   AS is_current
FROM silver_schema.product_silver
WHERE 1 = 0;
--Esto crea la tabla vacía con la estructura correcta.


In [0]:
%sql
MERGE INTO silver_schema.product_silver_scd2 t
USING src_product s
ON t.product_id = s.product_id
AND t.is_current = true

WHEN MATCHED 
AND t.record_hash <> s.record_hash
THEN UPDATE SET
  t.is_current = false,
  t.effective_to = current_timestamp();
/*
Si el producto existe
y estaba vigente
y cambió algo
👉 se “cierra” la versión anterior
*/


In [0]:
%sql
INSERT INTO silver_schema.product_silver_scd2
SELECT s.*
FROM src_product s
LEFT JOIN silver_schema.product_silver_scd2 t
  ON t.product_id  = s.product_id 
  AND t.is_current = true
WHERE t.product_id  IS NULL;
/*
Aquí entran:
productos nuevos
productos que cambiaron (nueva versión vigente)
*/


In [0]:
%sql
--Ver historial de un producto
SELECT product_id, effective_from, effective_to, is_current
FROM silver_schema.product_silver_scd2
WHERE product_id = 606
ORDER BY effective_from DESC;


In [0]:
%sql
--Ver historial de un producto
SELECT *
FROM silver_schema.product_silver_scd2
WHERE is_current = true;


In [0]:
%sql
--¿Hay data?
SELECT COUNT(*) AS total_rows
FROM silver_schema.product_silver_scd2;


In [0]:
%sql
--¿Cuántos vigentes?
SELECT COUNT(*) AS current_rows
FROM silver_schema.product_silver_scd2
WHERE is_current = true;


In [0]:
%sql
--Validación clave: no debe haber 2 vigentes por PK debe de haber 0
SELECT product_id, COUNT(*) AS current_versions
FROM silver_schema.product_silver_scd2
WHERE is_current = true
GROUP BY product_id
HAVING COUNT(*) > 1;


**Prueba de “cambio” (Update) — la prueba más importante**

In [0]:
%sql
--Elegir un producto (uno cualquiera)
SELECT product_id
FROM silver_schema.product_silver
LIMIT 10;


In [0]:
%sql
--Mira su historial antes
SELECT product_id, effective_from, effective_to, is_current
FROM silver_schema.product_silver_scd2
WHERE product_id = 463
ORDER BY effective_from DESC;


**Simular un cambio en product_silver**

In [0]:
%sql
--Realizamos cambio de un color
UPDATE silver_schema.product_silver
SET Color = 'TEST_COLOR_SCD2'
WHERE product_id = 463;


**Ejecutamos el notebook SCD2 nuevamente**

In [0]:
%sql
SELECT product_id, Color, effective_from, effective_to, is_current
FROM silver_schema.product_silver_scd2
WHERE product_id = 463
ORDER BY effective_from DESC;
